# TP1 : introduction à JuMP

[JuMP](https://github.com/JuliaOpt/JuMP.jl) est un package Julia permettant la modélisation de programmes mathématiques, notamment des problèmes d'optimisation linéaire. C'est également une interface pour les principaux solveurs d'optimisation tels que CBC, CLP, Glpk, Cplex, Gurobi, etc.

Ce TP a pour objectif de se familiariser avec ce package.

Voici un exemple de problème d'optimisation linéaire modélisé avec le package JuMP et résolu avec le solveur GLPK. Répondre aux questions suivantes :

1. Éxécuter le code pour déterminer la solution optimale.
2. Quelle est la solution optimale si l'objectif est maintenant minimiser 18x - 3y ?
3. Ajouter la contrainte $2x + y \ge 4$. Cela change-t-il l'optimum ?
4. Modifier le membre de droite de la nouvelle contrainte par 9. Que se passe-t-il ?

In [1]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

In [2]:
# Création d'un modèle. Ce modèle fera l'interface avec le solveur GLPK
m = Model(GLPK.Optimizer)

#Création d'une variable x dans le modèle m. x est une variable continue positive ou nulle.
@variable(m, 0 <= x)

#Création d'une deuxième variable continue y entre 0 et 30
@variable(m, 0 <= y <= 30 )

#Création de la fonction objective du modèle
#=
ATTENTION : 2 écritures possibles : 5x + 3y ou 5 * x + 3 * y.
            Par contre, 5 x + 3 y produit une erreur
=#
@objective(m, Max, 4x + 5y)

#Ajout d'une contrainte dans le modèle
@constraint(m, x + 5y <= 3.5 ) # == # >=


#Affichage du modèle
print(m)

#Résolution du problème d'optimisation linéaire m par le solveur GLPK
optimize!(m)

status = termination_status(m)


if status == INFEASIBLE
    println("Le problème n'est pas réalisable")
elseif status == UNBOUNDED
    println("Le problème est non borné")
elseif status == OPTIMAL
    println("Optimum = ", JuMP.objective_value(m))
    println("Solution optimale :")
    println("\t x = ", JuMP.value(x))
    println("\t y = ", JuMP.value(y))
else
    println("Problème lors de la résolution")
end

Optimum = 14.0
Solution optimale :
	 x = 3.5
	 y = 0.0


## Exercice 1 : Résolution d'un PLNE

Résoudre le PLNE suivant :

$\min 78x + 25y - 54z$

$27x - 12y + 4z \geq 140$

$20y + 47z = 120$

$x \in \{0,1\}$

$y \text{ entier}$

$z \geq 0$

**Remarque :** On peut définir des variables binaires ou entières en indiquant lors de la création de la variable son type `@variable(m, nom_variable, type)` où le type peut être `Int` ou `Bin`.

Par exemple : `@variable(m, x, Bin)` ajoute une variable `x` binaire au modèle `m`.


In [2]:
############################## 
#   Saisir votre code ici.   #
##############################
# Création d'un modèle. Ce modèle fera l'interface avec le solveur GLPK
m = Model(GLPK.Optimizer)

#Création d'une variable x dans le modèle m. x est une variable continue positive ou nulle.
@variable(m, 0 <= x,Bin)

#Création d'une deuxième variable continue y entre 0 et 30
@variable(m, y, Int)
@variable(m, 0 <= z)

#Création de la fonction objective du modèle
#=
ATTENTION : 2 écritures possibles : 5x + 3y ou 5 * x + 3 * y.
            Par contre, 5 x + 3 y produit une erreur
=#
@objective(m, Min, 78x + 25y - 54z)

#Ajout d'une contrainte dans le modèle
@constraint(m, 27x - 12y +4z >= 140 ) # == # >=
@constraint(m, 20y + 47z == 120 ) # == # >=


#Affichage du modèle
print(m)

#Résolution du problème d'optimisation linéaire m par le solveur GLPK
optimize!(m)

status = termination_status(m)


if status == INFEASIBLE
    println("Le problème n'est pas réalisable")
elseif status == UNBOUNDED
    println("Le problème est non borné")
elseif status == OPTIMAL
    println("Optimum = ", JuMP.objective_value(m))
    println("Solution optimale :")
    println("\t x = ", JuMP.value(x))
    println("\t y = ", JuMP.value(y))
    println("\t z = ", JuMP.value(z))
else
    println("Problème lors de la résolution")
end

Le problème est non borné



## Exercice 1 : Fabrique de fibre optique


Une entreprise de fabrication de fibre optique doit décider de son plan de production pour la semaine suivante. Elle peut fabriquer trois types de fibre : 
 * Fibre Fluorée vendue à 35 &euro; le mètre
 * Fibre Chlorée vendue à 24 &euro; le mètre
 * Fibre Mélangée vendue à 39 &euro; le mètre

La fabrication de chaque type de fibre nécessite différentes ressources résumées dans le tableau ci-dessous.

| Ressources    | Fibre Fluorée  | Fibre Chlorée    | Fibre Mélangée    |
| ------------- | -------------: | ---------: | ---------: |
| Silice        | 10 kg/m        | 15 kg/m    | 15 kg/m    |
| Fluor         | 20 kg/m        | 0  kg/m    | 12 kg/m    |
| Chlore        | 0 kg/m         | 34 kg/m    | 18 kg/m    |
| Main-d'oeuvre | 3 h/m          | 4 h/m      | 6 h/m      |

L'entreprise possède une tonne de silice, 400kg de Fluor, 350kg de Chlore. Par ailleurs, elle dispose d'une équipe de 7 ouvriers pouvant fabriquer les différents types de fibres optiques, chacun des ouvriers travaillant 35h par semaine. Elle souhaite maximiser son profit généré par la vente de ses différentes fibres (on suppose que l'entreprise peut vendre tout ce qu'elle produit. De plus, elle peut produire n'importe quelle quantité (même quelques microns de fibre optique).

Modéliser le problème avec JuMP puis déterminer le plan de production optimal.

In [3]:
##################
#   Correction   #
##################


# Création d'un modèle. Ce modèle fera l'interface avec le solveur GLPK
m = Model(GLPK.Optimizer)

@variable(m,  F >= 0) # Quantité de fibre fluorée
@variable(m,  C >= 0) # Quantité de fibre chlorée
@variable(m,  M >= 0) # Quantité de fibre mélangée


#Création de la fonction objective du modèle
@objective(m, Max, 35F + 24C  + 39M)

#Ajout des contraintes
@constraint(m, 10F + 15C + 15M <= 1000 ) # Ressource silice
@constraint(m, 20F + 12M <= 400 ) # Ressource fluor
@constraint(m, 34C + 18M <= 350 ) # Ressource chlore
@constraint(m,  3F +  4C + 6M <= 7 * 35) # Main d'oeuvre

# Résolution du problème d'optimisation linéaire m par le solveur GLPK
optimize!(m)

# On vérifie que le problème est résolu à l'optimum
status = termination_status(m)
@assert status == OPTIMAL

println("Profit = ", JuMP.objective_value(m), "€")
println("Solution optimale :")

println("\t Quantité de fibre fluorée = ", JuMP.value(F), "m")
println("\t Quantité de fibre chlorée = ", JuMP.value(C), "m")
println("\t Quantité de fibre mélangée = ", JuMP.value(M), "m")


Profit = 1050.0€
Solution optimale :
	 Quantité de fibre fluorée = 8.333333333333334m
	 Quantité de fibre chlorée = 0.0m
	 Quantité de fibre mélangée = 19.444444444444443m
